# **Log into the google drive to access the uploaded data**

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd gdrive/MyDrive/Courses/RE/project_group_4/data

# %cd gdrive/MyDrive/data

# !echo 'load complete!'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/MyDrive/Courses/RE/project_group_4/data'
/content/gdrive/MyDrive/Courses/RE/project_group_4/data
load complete!


# -1: Environment Setup

### Install Dependencies

In [2]:
!pip install pandas~=1.4.3 numpy~=1.23.1 nltk~=3.7 gensim~=4.2.0 textblob~=0.17.1 scikit-learn~=1.1.1 hypy_utils~=1.0.10

In [115]:
import csv
import os
import pandas as pd
import xml.etree.ElementTree as ET
import warnings
warnings.filterwarnings('ignore')

f = open('stopwords.txt', 'r')
stopwords = f.read().splitlines()

all_candidate_links = 0
issues_total = 0
commits_total = 0
reviews_total = 0
links_total = 0
#iterate though our 10 app dirs
for dir_cnt, dir in enumerate(os.listdir('.')):
  if dir == 'test' or dir == 'stopwords.txt': continue
  # if dir_cnt == 1: break
  # dir = 'ics-openconnect'
  print('\n\nProject:', dir)

  # open strings.xml file and get gui keywords
  gui_keywords = []
  tree = None
  try:
    tree = ET.parse(dir+'/strings.xml')
  except:
    print('Cannot open strings.xml!')
  root = tree.getroot()
  for el in root.iter('string'):
    # print(el.attrib['name'])
    for gui in el.attrib['name'].split('_'):
      if gui not in gui_keywords:
        gui_keywords.append(gui)
  
  # discarding gui keywords in stopwords
  stop = []
  for sw in stopwords:
    if sw in gui_keywords:
      continue
    else:
      stop.append(sw)
  

  # open each files [review, issue, commit, commit_files]
  try:
    issue_df = pd.read_csv(os.path.join(dir, dir + '_issue.csv'), encoding = 'ISO-8859-1')
  except:
    print('Cannot open', dir+'_issue', '!!!')
  try:
    commit_df = pd.read_csv(os.path.join(dir, dir + '_commit.csv'), encoding = 'utf-8-sig', sep='@@@', engine='python')
  except :
    print('Cannot open', dir+'_commit', '!!!')
  try:
    review_df = pd.read_csv(os.path.join(dir, dir + '_review.csv'), encoding = 'ISO-8859-1')
  except:
    print('Cannot open', dir+'_review', '!!!')
  try:
    commit_files_df = pd.read_csv(os.path.join(dir, dir + '_commit_files.csv'), encoding = 'ISO-8859-1')
  except:
    print('Cannot open', dir+'commit_files !!!')


  # only get informative reviews
  # print('all_reviews', review_df.shape)
  review_df = review_df.loc[review_df['Class'] == "[{'term':'Informative'}]"]
  # print data info
  # print('informative_reviews', review_df.shape)
  # print('commits', commit_df.shape)
  # print('issues', issue_df.shape)

  # remove stop words
  issue_df['Issue Name'] = issue_df['Issue Name'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
  issue_df['Issue Description'] = issue_df['Issue Description'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop)]))
  commit_df['Commit Message'] = commit_df['Commit Message'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
  review_df['User Review'] = review_df['User Review'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop)]))

  # normalize date format
  issue_df['Date Opened'] = pd.to_datetime(issue_df['Date Opened'], errors='coerce')
  issue_df['Date Closed'] = pd.to_datetime(issue_df['Date Closed'], errors='coerce')
  review_df['Date'] = pd.to_datetime(review_df['Date'], errors='coerce', utc=True)
  commit_df['Date'] = pd.to_datetime(commit_df['Date'], errors='coerce', utc=True)
  review_df['Date'] = review_df['Date'].dt.date
  commit_df['Date'] = commit_df['Date'].dt.date

  # Add file names to commit message
  commit_files_df['CommitID'] = commit_files_df['CommitID'].str[:7]
  for i, commit_file in commit_files_df.iterrows():
    for j, commit in commit_df.iterrows():
      if commit_file['CommitID'] == commit['Commit ID']:
        file_list = []
        for each_file in commit_file[' List of files'].split():
          file_name = each_file.split('/')[-1]
          try:
            file_name = file_name.split('.')[-2].casefold()
          except:
            file_name = file_name.casefold()
          file_list.append(file_name)
        commit['Commit Message'] += ' '.join(file_list)


  reviews_total += review_df.shape[0]
  issues_total += issue_df.shape[0]
  commits_total += commit_df.shape[0]
  print('informative reviews:', review_df.shape)
  print('issues:', issue_df.shape)
  print('commits:', commit_df.shape)


  # ISSUE and COMMIT EXTRACTOR
  # link candidate issues and commit to reviews (posted after review) 
  review_and_issue = []
  review_and_commit = []
  for i, review in review_df.iterrows():
    for j, issue in issue_df.iterrows():
      if review['Date'] < issue['Date Opened']:
        review_and_issue.append([review, issue])
    for j, commit in commit_df.iterrows():
      if review['Date'] < commit['Date']:
        review_and_commit.append([review, commit])

  print('ISSUE EXTRACTOR:', len(review_and_issue))
  print('COMMIT EXTRACTOR:', len(review_and_commit))
  print()
  
  # LINK IDENTIFIER

  # links issues to reviews wrt similarity score
  threshold = 0.3
  r_i_header = review_and_issue[0][0].to_frame().T.columns.to_list() + review_and_issue[0][1].to_frame().T.columns.to_list()
  temp = []
  for i, r_i in enumerate(review_and_issue):
    a_review = r_i[0]['User Review'].split()
    a_review = [x.casefold() for x in a_review]
    a_issue = r_i[1]['Issue Name'].split() + str(r_i[1]['Issue Description']).split()
    a_issue = [x.casefold() for x in a_issue]

    if len(a_review) == 0 or len(a_issue) == 0:
      continue
    text_sim = len(set(a_review).intersection(set(a_issue))) / min(len(a_review), len(a_issue))
    
    # compute GUI bonus
    gui_ir = []
    for i_r in a_review:
      if i_r in gui_keywords:
        gui_ir.append(i_r)
    gui_is = []
    for i_s in a_issue:
      if i_s in gui_keywords:
        gui_is.append(i_s)
    union = len(set(a_review).union(set(a_issue)))
    gui_bonus = len(set(gui_ir).intersection(set(gui_is))) / union
    
    final_sim = text_sim * 0.5 + gui_bonus * 0.5
    if final_sim > threshold and len(a_review) > 3:
      link = list(pd.concat([r_i[0], r_i[1]]))
      temp.append(link)
      print(link)
      links_total += 1
    all_candidate_links += 1
  review_issue_linked = pd.DataFrame(temp, columns=r_i_header)


  # links commits to reviews
  r_c_header = review_and_commit[0][0].to_frame().T.columns.to_list() + review_and_commit[0][1].to_frame().T.columns.to_list()
  temp = []
  for i, r_c in enumerate(review_and_commit):
    a_review = r_c[0]['User Review'].split()
    a_review = [x.casefold() for x in a_review]
    a_commit = r_c[1]['Commit Message'].split()
    a_commit = [x.casefold() for x in a_commit]

    if len(a_review) == 0 or len(a_issue) == 0:
      continue
    text_sim = len(set(a_review).intersection(set(a_commit))) / min(len(a_review), len(a_commit))

    # compute GUI bonus
    gui_ir = []
    for i_r in a_review:
      if i_r in gui_keywords:
        gui_ir.append(i_r)
    gui_ic = []
    for i_c in a_commit:
      if i_c in gui_keywords:
        gui_ic.append(i_c)
    union = len(set(a_review).union(set(a_commit)))
    gui_bonus = len(set(gui_ir).intersection(set(gui_ic))) / union

    # compute the final similarity score
    final_sim = text_sim * 0.5 + gui_bonus  * 0.5
    if final_sim > threshold and len(a_review) > 3:
      link = list(pd.concat([r_c[0], r_c[1]]))
      temp.append(link)
      print(link)
      links_total += 1
    all_candidate_links += 1
  review_commit_linked = pd.DataFrame(temp, columns=r_c_header)
  review_commit_linked.to_csv('../results/'+dir+'_review2commit.csv')
  review_issue_linked.to_csv('../results/'+dir+'_review2issue.csv')


print('\n\nall_candidate_links', all_candidate_links)
print('total reviews:', reviews_total)
print('total issues:', issues_total)
print('total commits:', commits_total)
print('total links:', links_total)




Project: ca.rmen.android.frenchcalendar
informative reviews: (23, 5)
issues: (30, 8)
commits: (345, 5)
ISSUE EXTRACTOR: 392
COMMIT EXTRACTOR: 3532



Project: ics-openconnect
informative reviews: (216, 5)
issues: (64, 8)
commits: (935, 5)
ISSUE EXTRACTOR: 12686
COMMIT EXTRACTOR: 6505

['Connect with office VPN', "[{'term':'Informative'}]", 0.5, 5, datetime.date(2014, 2, 21), 23, 'Cannot connect to non-standard HTTPS', Timestamp('2017-09-07 00:00:00'), nan, 'peloy', Timestamp('2019-06-05 00:00:00'), 'dlenski', 'title: Cannot connect to non-standard HTTPS state: CLOSED author: peloy labels: comments: 7 assignees: projects: milestone: number: 23 -- My Cisco ASA configured to terminate SSL VPN connection 4443. Android OpenConnect (latest version from the Google Play store) not able to connect. The OpenConnect log the screenshot mentioned the ASA apparently returning the message "Invalid host entry. Please re-enter". OpenConnect 7.08 running Linux able to connect with no issues. On the AS

In [116]:
issue_df.head()

,Issue Number,Issue Name,Date Opened,Version,Issue Author,Date Closed,Closed By,Issue Description
0,1,[Feature] Profiles,2014-05-30,2.0,corcoran,NaT,Not closed,title: [Feature] Profiles state: OPEN author: ...
1,2,[Feature] Custom icon packs,2014-06-12,2.1,corcoran,2014-07-16,corcoran,title: [Feature] Custom icon packs state: CLOS...
2,3,[Feature] Time periods in-app,2014-06-16,2.1,corcoran,NaT,Not closed,title: [Feature] Time periods in-app state: OP...
3,4,[Feature] Pinned apps,2014-07-16,2.2,corcoran,NaT,Not closed,title: [Feature] Pinned apps state: CLOSED aut...
4,5,[Bug] Option to download icon pack disappears,2014-07-23,2.2,DarkDrek,NaT,Not closed,title: [Bug] Option to download icon pack disa...


In [ ]:
review_df.head()

,User Review,Class,Version,Rating,Date
0,Wasnt I I downloaded pretty cool. I replace no...,[{'term':'Informative'}],2.0,5,2014-05-17
1,"After couple days hangar, I imagine not intell...",[{'term':'Informative'}],2.0,5,2014-05-18
2,A whitelist useful blacklist.,[{'term':'Informative'}],2.0,5,2014-05-29
4,All space notification area going to waste. No...,[{'term':'Informative'}],2.1,5,2014-07-06
5,"Great stylish app, low resource usage, virtual...",[{'term':'Informative'}],2.1,5,2014-07-06


In [ ]:
commit_df.head()


,Commit ID,Commit Message,Author,Email,Date
0,879b544,Update README.mdreadme,Jeff Corcoran,jcorcoran+github@gmail.com,2019-09-25
1,ed4e9c1,Merge pull request #25 from rzabcik/master,Jeff Corcoran,jcorcoran+github@gmail.com,2019-09-25
2,f6c868a,Merge pull request #40 from Poussinou/patch-1,Jeff Corcoran,jcorcoran+github@gmail.com,2019-09-25
3,1.66E+07,Update README.md,Poussinou,jcorcoran+github@gmail.com,2019-09-21
4,3be6b4f,crash on launch caused by nullptr invalid icon...,Ryan Zabcik,jcorcoran+github@gmail.com,2017-07-15
